## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [19]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,53.10,75,0,8.39,clear sky
1,1,Ushuaia,AR,-54.8000,-68.3000,33.46,93,75,23.02,light snow
2,2,Butaritari,KI,3.0707,172.7902,80.40,72,98,7.92,overcast clouds
3,3,Dikson,RU,73.5069,80.5464,21.79,93,92,5.75,overcast clouds
4,4,Nemuro,JP,43.3236,145.5750,50.81,97,69,13.91,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip?'))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))

In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
(city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Butaritari,KI,3.0707,172.7902,80.40,72,98,7.92,overcast clouds
5,5,Avarua,CK,-21.2078,-159.7750,82.45,74,38,10.36,scattered clouds
6,6,Bambous Virieux,MU,-20.3428,57.7575,70.21,94,20,6.91,few clouds
9,9,Vila Velha,BR,-20.3297,-40.2925,71.94,79,29,0.94,scattered clouds
11,11,San Andres,CO,12.5847,-81.7006,81.77,81,100,15.10,overcast clouds
13,13,Olinda,BR,-8.0089,-34.8553,77.14,83,44,16.28,scattered clouds
16,16,Minab,IR,27.1467,57.0801,85.68,48,0,5.84,clear sky
21,21,Lianzhou,CN,24.7811,112.3825,72.25,99,100,1.57,overcast clouds
22,22,Laguna,US,38.4210,-121.4238,86.14,42,0,21.85,clear sky
26,26,Rikitea,PF,-23.1203,-134.9692,77.25,74,23,17.13,few clouds


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                273
City                   273
Country                273
Lat                    273
Lng                    273
Max Temp               273
Humidity               273
Cloudiness             273
Wind Speed             273
Current Description    273
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#Not Applicable

In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Butaritari,KI,80.40,overcast clouds,3.0707,172.7902,
5,Avarua,CK,82.45,scattered clouds,-21.2078,-159.7750,
6,Bambous Virieux,MU,70.21,few clouds,-20.3428,57.7575,
9,Vila Velha,BR,71.94,scattered clouds,-20.3297,-40.2925,
11,San Andres,CO,81.77,overcast clouds,12.5847,-81.7006,
13,Olinda,BR,77.14,scattered clouds,-8.0089,-34.8553,
16,Minab,IR,85.68,clear sky,27.1467,57.0801,
21,Lianzhou,CN,72.25,overcast clouds,24.7811,112.3825,
22,Laguna,US,86.14,clear sky,38.4210,-121.4238,
26,Rikitea,PF,77.25,few clouds,-23.1203,-134.9692,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location']=f'{lat},{lng}'
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json() 
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
# 7. Drop the rows where there is no Hotel Name.
nill = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = nill.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Butaritari,KI,80.40,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
5,Avarua,CK,82.45,scattered clouds,-21.2078,-159.7750,Paradise Inn
6,Bambous Virieux,MU,70.21,few clouds,-20.3428,57.7575,Casa Tia Villa
9,Vila Velha,BR,71.94,scattered clouds,-20.3297,-40.2925,Hotel Vitória Palace
11,San Andres,CO,81.77,overcast clouds,12.5847,-81.7006,Hotel Arena Blanca
...,...,...,...,...,...,...,...
662,Cockburn Town,TC,79.84,few clouds,21.4612,-71.1419,Osprey Beach Hotel
663,Gerash,IR,78.49,clear sky,27.6652,54.1371,Sa'adat House Gerash
665,Sambava,MG,71.46,scattered clouds,-14.2667,50.1667,Melrose
670,Mozarlandia,BR,75.94,clear sky,-14.7447,-50.5706,Hotel Santa Mônica


In [38]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [39]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [41]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))